@YoussefM

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import re

In [2]:
import pandas as pd

In [49]:
def poets_per_page(i: int):
    """
    takes a page number, an integer between 1 and 188, and returns the list 
    of all poets in the given page 
    """
    poets = []
    address = 'https://diwandb.com/poets?page='+str(i)
    url = requests.get(address)
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    data = soup.find_all(href=re.compile("/poet/"))
    for poet in data:
        poet_ = 'https://diwandb.com'+poet.get('href')
        
        page_poet = requests.get(poet_).content
        soup_poet = BeautifulSoup(page_poet, "lxml")
        poet_card = soup_poet.find("div", {'class': 'overflow-y-scroll px-4 py-6 max-h-[70vh] text-sm bg-white dark:bg-gray-800 rounded shadow-lg text-base'})
         
        poets += [[poet_, poet_card.text]]
        
        
    return poets

In [105]:
# get the list of all poets in the website, that's 4,512 poet. 
from tqdm.notebook import tqdm
all_poets = []
for i in tqdm(range(1,189)):
    all_poets += poets_per_page(i)

  0%|          | 0/188 [00:00<?, ?it/s]

We'll create a csv file to contain the poets with their biographie.

In [106]:
dict = {'poet': [all_poets[i][0] for i in range(len(all_poets))], 'bio': [all_poets[i][1] for i in range(len(all_poets))]}

In [107]:
pd.DataFrame(dict).to_csv('all_poets')

In [4]:
all_poets = pd.read_csv("all_poets")

In [112]:
all_poets

,Unnamed: 0,poet,bio
0,0,https://diwandb.com/poet/ابن-الرومي.html,\nابن الرومي221 - 283 هـ / 836 - 896 معلي بن ا...
1,1,https://diwandb.com/poet/أبو-العلاء-المعري.html,\nأحمد بن عبد الله بن سليمان، التنوخي المعري. ...
2,2,https://diwandb.com/poet/ابن-نباتة-المصري.html,\nمحمد بن محمد بن محمد بن الحسن الجذامي الفارق...
3,3,https://diwandb.com/poet/عبد-الغني-النابلسي.html,\nعبد الغني النابلسي1050 - 1143 هـ / 1641 - 17...
4,4,https://diwandb.com/poet/جبران-خليل-جبران.html,\nجبران خليل جبران ولد في 6 يناير 1883 في بلدة...
...,...,...,...
4507,4507,https://diwandb.com/poet/أبو-دهمان-محمد-بن-أبي...,\nأَبُو دهمان بصرى عَرَبِيّ تقلد سَابُور من كو...
4508,4508,https://diwandb.com/poet/دينار-بن-نعيم.html,\nدينار بن نُعَيْم دينار بن نعيم بن حصين بن سع...
4509,4509,https://diwandb.com/poet/ابن-جرار-الحمي.html,\nابن جرار الحمي أبو عبد الله محمد بن عيسى بن ...
4510,4510,https://diwandb.com/poet/ابن-الباطوخ-الواعظ.html,\n(ت 544 ه) مُحَمَّد بن مُحَمَّد بن عليّ ابْن ...


In [124]:
# the biography of Ibn Rumi for example (https://en.wikipedia.org/wiki/Ibn_al-Rumi)
all_poets['bio'][0]

'\nابن الرومي221 - 283 هـ / 836 - 896 معلي بن العباس بن جريج أو جورجيس، الرومي.شاعر كبير، من طبقة بشار والمتنبي، رومي الأصل، كان جده من موالي بني العباس.ولد ونشأ ببغداد، ومات فيها مسموماً قيل: دس له السمَّ القاسم بن عبيد الله -وزير المعتضد- وكان ابن الرومي قد هجاه.قال المرزباني: لا أعلم أنه مدح أحداً من رئيس أو مرؤوس إلا وعاد إليه فهجاه، ولذلك قلّت فائدته من قول الشعر وتحاماه الرؤساء وكان سبباً لوفاته.وقال أيضاً: وأخطأ محمد بن داود فيما رواه لمثقال (الوسطي) من أشعار ابن الرومي التي ليس في طاقة مثقال ولا أحد من شعراء زمانه أن يقول مثلها إلا ابن الرومي.\n'

The next two functions will be used to define `poems_per_poet(poet_url)` below

In [117]:
def get_titles_per_page(address):
    titles_per_page = []
    url = requests.get(address)
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    # list of all the titles in the page (address)
    frames = soup.find_all("div", {'class': 'slice'})
    for i in range(len(frames)):
        titles_per_page += ['https://diwandb.com'+frames[i].find(href=re.compile("/poem/")).get('href')]
    return titles_per_page

# the output of this function will be fed to get_poem_info()  

In [118]:
get_titles_per_page('https://diwandb.com/poet/%D8%B9%D9%84%D9%8A-%D8%A8%D8%A7.html?page=2')[0:5]

['https://diwandb.com/poem/أحد-عشر-كوكبا.html',
 'https://diwandb.com/poem/ليلة-الغرباء.html',
 'https://diwandb.com/poem/فبدأت-حيث-توقفوا.html',
 'https://diwandb.com/poem/أسير-هموم.html',
 'https://diwandb.com/poem/فقد-الأحبة-غربة.html']

In [119]:
def pages_per_poet(poet_url: str)->list:
    """
    Given a poet, this function will return all the pages of his/her poems
    """
    all_pages = []
    url = requests.get(poet_url.split('?')[0])
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    pages = soup.find("nav", {'class': 'pagination'})
    if pages==None:
        return poet_url
    i = 0
    while pages.find('button', {'wire:click': "nextPage"}):
        poet_url = poet_url.split('?')[0] + f"?page={i+1}"
        all_pages += [poet_url]
        url = requests.get(poet_url)
        page = url.content
        soup = BeautifulSoup(page, "lxml")
        pages = soup.find("nav", {'class': 'pagination'})
        i +=1
    return all_pages
    
# the output of this function will be fed to get_titles_per_page()

This is an important function. Given the url of a poet, it returns the list of his/her poems

In [120]:
def poems_per_poet(poet_url):
    """
    returns the poems of a given author
    """
    poems = []
    for address in pages_per_poet(poet_url):
        poems += get_titles_per_page(address)
    return poems

In [121]:
poet_url = 'https://diwandb.com/poet/%D8%B5%D9%82%D8%B1-%D8%A3%D8%A8%D9%88-%D8%B9%D9%8A%D8%AF%D8%A9.html'
poems_per_poet(poet_url)[0:5]

['https://diwandb.com/poem/عائشة-وحي-القصيد.html',
 'https://diwandb.com/poem/بوح-الأوردة.html',
 'https://diwandb.com/poem/تقاسيم-على-ثوب-الفجر.html',
 'https://diwandb.com/poem/لمن-يبكي.html',
 'https://diwandb.com/poem/النحت-في-صخر-الحروف.html']

Feeding all_poets to the function `poems_per_poet(poet_url)` above, we get all the (urls of) poems published in the website. Then to each of these, we apply the next function to get the  poem itself, its `period` and `meter`. 

In [122]:
def get_poem_info(poem_url: str):
    """
    the function gets the url address and returns the info of the poem
    """
    url = requests.get(poem_url)
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    # the body of the poem
    poem = soup.find("div", {'class': 'mx-auto w-full text-xl sm:text-3xl md:max-w-lg twice leading-12'})
    # the title
    title = soup.find("div", {'class': 'mb-6'}).find("h1").text
    # next we extract the name of the poet, the meter and the historical period
    data = soup.find("div", {'class': 'flex flex-wrap justify-between items-center my-4 leading-10'})
    poet = data.find(href=re.compile("poet")).text.strip('\n')
    meter = data.find(href=re.compile("meter")).text.strip('#')
    period = data.find(href=re.compile("period")).text.strip('#')